# 2. Textklassifikation


### a.
Die Textdatenbank20 Newsgroupsfindet sich zum Download unter http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz. Benutzen Sie zum Entpacken das Python-Standardpaket tarfile. Die Datenbank enthält ca. 20.000 Textdokumente, etwa gleichmäßig aufgeteilt auf 20 Newsgroups zu unterschiedlichen Diskussionsthemen. Die Dokumente zu jeder Newsgroup befinden sich nach dem Entpacken in einem Verzeichnis gleichen Namens.

In [ ]:
import os
from os.path import join
import tarfile

# Open Tarfile
tar = tarfile.open("C:/Users/morit/iCloudDrive/MWI_3/Maschinelles_Lernen/ML_A2/20news-18828.tar.gz")
# Extract Tarfile if not available
if not "C:/Users/morit/iCloudDrive/MWI_3/Maschinelles_Lernen/ML_A2/20news-18828":
    tar.extractall()

### b.
Wir beschränken uns auf die vier Newsgroups alt.atheism, comp.graphics, sci.space und talk.religion.misc.  
Lesen Sie alle Dateien aus diesen vier Verzeichnissen in eine Array von Strings ein (d.h. ein File in einem String). Speichern Sie zusätzlich die Klassenzugehörigkeit jedes Dokuments in einem Vektor ab (Kontrolle: Sie müssten jetzt 3387 Stringsim Speicher haben). 

In [ ]:
# list for file data
data = []
# list for file names
list_files = []
# list for file classes
classes = []
# relevant newsgroups
newsgroup = ["alt.atheism", "comp.graphics", "sci.space", "talk.religion.misc"]
# path to dir
path = "C:/Users/morit/iCloudDrive/MWI_3/Maschinelles_Lernen/ML_A2/20news-18828"

# iterate through filenames(directories) in dir
for filename in os.listdir(path):
    # choose relevant directories
    if filename in newsgroup:
        path_to_dir = os.path.join(path, filename)
        # iterate through files in relevant dir
        for file in os.listdir(path_to_dir):
            path_to_file = os.path.join(path_to_dir, file)
            # open file, read file, append to data list, close file
            fd = os.open(path_to_file, os.O_RDONLY)
            data.append(os.read(fd, 1000000).decode("latin-1"))
            os.close(fd)
            # append file name to list for file names
            list_files.append(file)
            classes.append(filename)

print(len(data))


Ungünstigerweise enthalten diese Dokumente immer noch den Namen der Newsgroup im Header, was eine Klassifikation trivial machen würde. Wir müssen also die ersten 2 Zeilen aus jedem Dokument entfernen, was mit folgendem Code geschieht (die Dokumente befinden sich als Strings im Array data):

In [ ]:
def strip_header(text):
    _before , _blankline , after = text.partition("\n\n")
    return after

data = [strip_header(text) for text in data]

### c.
Im nächsten Schritt muss jeder String in Worte (Tokens) zerlegt werden, die durch Leerzeichen, Kommas etc. voneinander getrennt sind. Hierzu setzen wir das Python-Standardpaket re ein, das zur Analyse regulärer Ausdrücke dient. Durch folgenden Befehl werden alle Tokens eines Strings txtline in einer Liste l gespeichert, nachdem er zuvor mit lower() in Kleinbuchstaben umgewandelt wurde:

In [ ]:
import re
# list for all the tokens
l_all = []

for txtline in data:
    l = re.compile(r"(?u)\b\w\w+\b").findall(txtline.lower())
    for word in l:
        l_all.append(word)
            
print(len(l_all))

unique_tokens = set(l_all)
len(unique_tokens)



Wer sich mit regulären Ausdrücken auskennt, kann hier mit anderen Suchmustern experimentieren. 
Bestimmen Sie auf diese Weise die Größe des gemeinsamen Vokabulars aller Texte (Kontrolle: mit obigem Suchmuster müssten Sie 41777 unterschiedliche Tokens erhalten, d.h. Ihre Merkmalsvektoren sind daher 41777-dimensional). Berechnen Sie für jeden Text einen Merkmalsvektor, der für jedes Wort des Vokabulars seine Häufigkeit innerhalb des Texts enthält.

In [ ]:
hot_encoder = [0] * len(unique_tokens)
binary_bag = dict(zip(unique_tokens, hot_encoder))
dict_all = dict(zip(list_files, data))
merkmalsvektoren = {}

for key in dict_all:
    binary_bag = dict(zip(unique_tokens, hot_encoder))
    l = re.compile(r"(?u)\b\w\w+\b").findall(dict_all[key].lower())
    
    for word in l:
        #if word in unique_tokens:
        binary_bag[word] += 1
    merkmalsvektoren[key] = binary_bag

print(len(merkmalsvektoren))

In [ ]:
import pandas as pd

df_files = pd.DataFrame({"id":list_files})
df_merkmalsvektoren = pd.DataFrame(data=merkmalsvektoren)

In [ ]:
df_mv_t = df_merkmalsvektoren.transpose()
df_classes = pd.DataFrame({"class":classes})
df_classes = df_classes.rename(columns={"class":"file_class"})

In [ ]:
display(df_classes.head())
display(df_mv_t.head())

### d.
Verwenden Sie die ersten 60% der Daten als Trainingsdatensatz, die restlichen als Testdatensatz. Trainieren Sie damit einen multinomialen naiven Bayes-Klassifikator. Bestimmen Sie den Anteil korrekter Klassifikationen auf Ihren Trainings- und Testdaten. Wie gut generalisiert Ihr Klassifikator?

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

X = df_mv_t.to_numpy()
y = df_classes.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state = 42)

clf = MultinomialNB()
clf.fit(X_train, y_train.ravel())
print(clf.score(X_test, y_test))
#print(clf.score(X_train, y_train))